<link rel="stylesheet" href="https://doc.splicemachine.com/zeppelin/css/zepstyles2.css" />

# Creating Custom Stored Procedures

This notebook describes how to create and use custom stored procedures written in Java in Splice Machine, in these sections:

* *Your Stored Procedure Development Environment*
* *Local Deployment of the Jar File*
* *Running Your Custom Procedure Locally*

You can create custom functions and procedures to deploy in your database; you can then call them as needed from your SQL.

<p class="noteIcon">This notebook describes how to create custom stored procedures. You will need to adapt this to your own development environment.</p>


## Your Stored Procedure Development Environment

The *environment* in which your stored procedure code is developed may change depending upon your need: we recommend that you flesh out the actual functionality of your custom code in a local (standalone) Splice Machine envirmonment. Once you have refined your code and it behaves to your satisfaction in your local environment, you can then re-deploy the developed code to your target instance. 

This notebook covers all of these steps - though as you will see, these steps occur OUTSIDE of an actual Notebook when running in a local environment. Replace any instances of `<your-file-name>` in the examples with your specific file name, rememberig to remove the `<` and `>` brackets.


## Local Development of the Custom Code in an IDE

As mentioned above, Splice Machine strongly recommends that you develop your custom Java code using an Integrated Development Environment (IDE) on your local machine. This is most frequently done on a Mac or Linux computer. That way you can develop, test, and tune locally, before deploying to a full cluster. The product of your code should be a JDK 1.8-compatible JAR file.

The following steps are for developing your procedure using the IntelliJ IDEA on MacOS:

### Setting Up Your Project

Inside of IntelliJ IDEA, do the following:
1. Select `File -> New -> Project...`
2. Choose `Java with 1.8` as the Project SDK
3. Skip creating from a template unless you prefer to use one
4. Give your project a name and location; for example: `CompanyACustom1`
5. If desired, create a new package (`File -> New -> Package`), such as `com.splicemachine.companya`

### Configuring to output a JAR file
Inside of IntelliJ IDEA, do the following:

1. Select `File -> Project Structure...`
2. Select `Artifacts`, and click `+`
3. Choose `JAR -> From modules with dependencies `
3. Check `Build on make` and then click `OK`

### Coding your function or procedure

1. Create a file by right-clicking on your src or package folder, and selecting `New -> Java Class`
2. Paste in code like the following:

```
package com.splicemachine.companya;

import java.sql.SQLException;
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.PreparedStatement;


public class Custom1 {


    public static void customIncrement () throws SQLException {
        try (Connection con = DriverManager.getConnection("jdbc:default:connection")) {

            String incrementSQL = "update SPLICE.INCREMENTTEST set i = i + 1";

            try (PreparedStatement ps = con.prepareStatement(incrementSQL)) {
                int resultSet = ps.executeUpdate();
            } catch (SQLException e) {
                con.rollback();
                e.printStackTrace();
                throw e;
            }
        }
    }
}
```

### Build your JAR file
1. `Build -> Artifacts... JAR`
2. Use your command line interface to navigate to where the JAR file is created, e.g. out/artifacts/CompanyACustom1_jar/Custom1.jar

You are now ready to deploy your JAR file to test!


## Development Guidelines
Before we develop the code and deploy the JAR file, it is worth going through some simple best practices:

1. It's useful for code organization to use a package for the code, e.g. com.splicemachine.yourcompanyname.
2. Notice you don't need a *main* code block in your class, because you'll be directly calling the class's specific method.
3. The example we've shown is a stored procedure, which means it does not return any results. When you create a function, it must return a result.
4. You need to ensure that your method throws `SQLException` if SQL is being executed.  Be sure to use `try/catch` blocks similar to those shown in the example, to deal with any possible run-time exceptions.


## Local Deployment of the JAR file

Once you've got your code up and running  you can deploy into your development environment. Carefully edit and execute the commands below IN YOUR LOCAL ENVIRONMENT (not in Zeppelin) to align to your custom procedure and environment:

```
CALL SQLJ.INSTALL_JAR('<your-full-path-to>/Custom1.jar', 'SPLICE.Custom1', 0);
CALL SYSCS_UTIL.SYSCS_SET_GLOBAL_DATABASE_PROPERTY('derby.database.classpath', 'SPLICE.custom1');
CREATE PROCEDURE SPLICE.customincrement() PARAMETER STYLE JAVA MODIFIES SQL DATA LANGUAGE JAVA DYNAMIC RESULT SETS 0 EXTERNAL NAME 'com.splicemachine.companya.Custom1.customIncrement';
```


## Run Your Procedure in Your Local Database Environment

You can create a table on which the stored procedure will operate, in your local environment. For example:
```
create table incrementtest (i int);
insert into incrementtest values 1;
```

And now you put your stored procedure into action:

```
call customincrement();
select * from incrementtest;
```


## Where to Go Next
Congratulations! You've just completed the *Splice Machine DBAs Class*. 

Visit [*Our Training Classes*](../About/Our%20Training%20Classes.ipynb) notebook learn about our other training classes.
